# Task description and pre-requisites:
## Should have a list of words at hand
- Some public list of all words in a language
    - Google NGrams - http://storage.googleapis.com/books/ngrams/books/datasetsv2.html
    - Project Gutenberg https://www.gutenberg.org/catalog/
- Words encountered in indexed documents (can also contain errors)
    - Manually add domain specific words
    
## Finding the correct word - Edit Distance
- Measure the number of single-character edits required to change one word into another
- Edits are: insert, delete, replace
- Solved with dynamic programming

# Edit Distance Algorithm:<br>
1. Initialize the matrix with zeros
2. Initialize 1st row and column with numbers from 0 to N and  from 0 to M
    * this is the cost of inserting a letter, when starting from the empty string
3. The value of each of the remaining cells is: <br>
<b>m[i, j] = min(<br>
            m[i-1,j] + 1, # insert in s1
            m[i,j-1] + 1, # insert in s2
            m[i-1,j-1] + (0 if s1[i]==s2[j] else 1) # replace
)</b> <br>
4. The min edit distance is at m[-1,-1]

| |S|A|T|U|R|D|A|Y|
-|-|-|-|-|-|-|-|-|-|
 |0|1|2|3|4|5|6|7|8|
S|1|0|1|2|3|4|5|6|7|
U|2|1|1|2|2|3|4|5|6|
N|3|2|2|2|3|3|4|5|6|
D|4|3|3|3|3|4|3|4|5|
A|5|4|3|4|4|4|4|3|4|
Y|6|5|4|4|5|5|5|4|3|

__Exercise 1.__ : Compute the edit distance between the words elephant and relevant. <br/>
__Exercise 2.__ : Implement the minimum edit distance algorithm.

Some corpora of misspellings can be found here: https://www.dcs.bbk.ac.uk/~ROGER/corpora.html

In [1]:
import numpy as np

def edit_distance(s1, s2):
    """
    Return the minumum edit distance between the two words
    """
    size_x = len(s1) + 1
    size_y = len(s2) + 1
    matrix = np.zeros ((size_x, size_y), dtype=int)
    for i in range(size_x):
        matrix[i, 0] = i
    for i in range(size_y):
        matrix[0, i] = i

    for i in range(1, size_x):
        for j in range(1, size_y):
            matrix[i,j] = min(matrix[i-1, j] + 1, matrix[i, j-1] + 1,
                             matrix[i-1, j-1] if s1[i-1] == s2[j-1] else matrix[i-1, j-1]+1)
    return (matrix[-1,-1])

In [2]:
edit_distance('lullaby', 'lollipop')

5

## Variants:
- Weighted Edit Distance
- Keeping a backtrack of the symbol alignments
- If a backtrack is not needed we can use only two matrix rows (memory reduced from O(nm) to O(2m))

## Some implementations:
- Levenstein Edit distance algorithm for Python in Cython for high performance 
    - https://github.com/gfairchild/pyxDamerauLevenshtein
- Peter Norvig's implementation
    - Idea : generate all possible words at edit distance 1 and 2, select those that are most probable (using a language model and priority of edit distance)
    - Can incorporate error model (some data with common errors http://aspell.net/test/)
    - http://norvig.com/spell-correct.html
- Library TextBlob - contains code for training a Spellchecker from text
    - https://github.com/sloria/TextBlob/blob/14f22102251ce1f02e8bcb3e74f86c037e3df822/textblob/_text.py#L1322

In [3]:
# !pip3 install textblob
from textblob import TextBlob
blob = TextBlob("I havv goood speling!")
blob.correct()

TextBlob("I have good spelling!")

In [4]:
#!pip3 install pyxDamerauLevenshtein
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
print(damerau_levenshtein_distance('smtih', 'smith'))
print(normalized_damerau_levenshtein_distance('smtih', 'smith'))

1
0.20000000298023224


# A Problem with the Edit Distance approach:
- Do we need to compute the edit distance to all dictionary terms? - __slow__
- Solution 1: 
    - build __charcter n-gram index__ - n-gram:word
        - Example: av -> avocado-> pavement, etc.
    - retrieve the postings of each n-gram in the query
    - merge postings, getting the words with most n-gram overlap
    - car can get corrected to carbon, then to normalize the metric for overlap we use __Jaccard coefficient__:
        - $ |\space X \cap Y \space|\space  /\space  |\space X \cup Y \space | $ 
        - If more than a threshold, then the words match
- Solution 2:
    - having to correct the word lates we will possibly compute edit distance to the words late, latest, latte
    - for each variant we will compute the edit distance of the common prefix late
    - rather use __tries__: store the whole vocabulary in a large trie, where one node is a symbol and the branches are the possible symbols following it
    - __compute the edit distance just once for the same prefix__!
    - Reference: http://stevehanov.ca/blog/index.php?id=114

__Exercise 3.__ Implement the n-gram overlap spell-checker. Find the best Jaccard coefficient for the chosen dataset. <br>
__Exercise 4.__ If you already decided what will be your project, build and tune your own spellchecker (or even one for Bulgarian!):
- Choose and implementation and a list of words (Textblob's spellchecker is pretrained on Gutenberg's data)
- Run your corpus through the spell-checker and find word, which were corrected, but are spelled correctly
- Add them to the corpus of words you chose

In [5]:
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
from os import scandir
tokenizer = TweetTokenizer()

def preprocess_documents(path):
    """
    Returns a list of tokens for a document's content. 
    Tokens should not contain punctuation and should be lower-cased.
    """
    tokenized_documents = []
    for doc_f in scandir(path):
        if not doc_f.is_file():
            continue
        content = open(doc_f).read()
        sentences = sent_tokenize(content)
        tokens = []
        for _sent in sentences:
            sent_tokens = tokenizer.tokenize(_sent)
            sent_tokens = [_tok.lower() for _tok in sent_tokens if _tok not in punctuation]
            tokens += sent_tokens
        tokenized_documents.append(tokens)
    return tokenized_documents

In [6]:
tokenized_documents = preprocess_documents('data/mini_newsgroups/rec.autos/')
tokenized_documents[0][:10]

['newsgroups',
 'rec.autos',
 'path',
 'cantaloupe.srv.cs.cmu.edu',
 'magnesium.club.cc.cmu.edu',
 'news.sei.cmu.edu',
 'fs7.ece.cmu.edu',
 'europa.eng.gtefsd.com',
 'howland.reston.ans.net',
 'ux1.cso.uiuc.edu']

In [7]:
from collections import defaultdict, Counter

def get_word_ngrams(word, n):
    return [word[i:i+n] for i in range(len(word)-n+1)]
        
        
def build_ngram_postings(n, tokenized_documents):
    """
    Create postings from the tokenized documents - 
    For each n-gram we want to have the list of words, containing that n-gram. (E.g. av: [avocado, pavement, ...])
    """
    processed_words = set()
    postings = defaultdict(lambda: [])
    for doc in tokenized_documents:
        for token in doc:
            if token in processed_words:
                continue
            else:
                processed_words.add(token)
                for ngram in get_word_ngrams(token, n):
                    postings[ngram].append(token)
    return postings

def ngram_spellcheck(word, postings, n, jaccard_threshold=0.7):
    """Return the word from the postings structure, which is closest to the input word."""
    ngram_postings = sum([postings[ngram] for ngram in get_word_ngrams(word, n)], [])
    suggestions = []
    not_close = 0
    for close_word, matching_ngrams in Counter(ngram_postings).most_common():
        jaccard_coeff = matching_ngrams / (len(close_word) - n + len(word) - n)
        if jaccard_coeff > jaccard_threshold:
            suggestions.append(close_word)
        else:
            not_close += 1
            if not_close > 10:
                break
    return suggestions

In [8]:
postings = build_ngram_postings(2, tokenized_documents)
print("Postings of bi-gram 'ar':")
print(postings['ar'][:10])
print("Closest words to enginering:")
print(ngram_spellcheck('enginering', postings, 2, jaccard_threshold=0.7))

Postings of bi-gram 'ar':
['article', 'darren', 'cars', 'hard', 'yard', 'compartment', 'are', 'early', 'area', 'antiauthoritarian']
Closest words to enginering:
['engineering', 'enginerring']


_Note: Building a Grammarly-like spelling/grammar correction for Bulgarian can be a good course project. 
If anyone is interested, we can discuss how to collect data and how to approach the task._